In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys 
base = '/home1/efeghhi/'
sys.path.append(f'{base}ripple_memory/analysis_code/')
from load_data import *
from analyze_data import *
sys.path.append(f'{base}ripple_memory/analysis_code/pac_analyses/')
from load_data_numpy import load_data_np
from comodulogram import remove_session_string, get_filtered_signal

import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import decimate, resample

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert
sys.path.append('/home1/efeghhi/ripple_memory/')
from SWRmodule import triangleSmooth


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def z_score(power):
    
    '''
    :param ndarray power: 2d array of shape trials x timesteps
    '''
    # mean center by mean across time and trials 
    # then divide by standard deviation of the average across timesteps
    power = (power - np.mean(power)) / np.std(np.mean(power, axis=1),axis=0)
    return power

def process_power(power, zscore_by_idxs):
    
    from scipy.signal import decimate
    
    power_ds = decimate(np.log10(power), 10)
    
    power_ds_zscored = np.zeros_like(power_ds)
    
    for val in np.unique(zscore_by_idxs):
        
        val_idxs = np.argwhere(zscore_by_idxs==val).squeeze()
            
        for freq in range(power_ds.shape[0]):
            
            power_ds_zscored[freq, val_idxs] = z_score(power_ds[freq, val_idxs].squeeze())
        
    return power_ds_zscored

In [4]:
# for now the plotting code only works with one region at a time
region_name = ['AMY'] 
subregion = ['']

# 1 for encoding, 0 for recall
encoding_mode_arr = [1] 
freq_range_str_arr = ['high', 'low']
savePath = f'{base}ripple_memory/analysis_code/pac_analyses/saved_results/power_figures'

# define frequency ranges
low_fq_range = [[2,4],[7,9]]
high_fq_range = [[30,70], [80,120], [130, 170]]


In [5]:
def load_z_scored_power(dd_trials, freq_range_str_arr):
    
    print("Generating figures for run_mode: ", encoding_mode)

    if encoding_mode:
        start_cutoff = 500
        end_cutoff = 2000

    else:
        start_cutoff = 500
        end_cutoff = 2500

    subjects = dd_trials['subj']

    fs = 500
    sr_factor = 1000/fs

    raw_data = dd_trials['raw']
    
    # z scoring is done for each unique subject, session, electrode combo
    subj_elec_sess_labels = dd_trials['elec_labels'] 

    for freq_range_str in freq_range_str_arr:
        
        if freq_range_str == 'high':
            freq_range = high_fq_range
            ylabel = 'Gamma power'
            bandwidth='auto'
        else:
            freq_range = low_fq_range
            ylabel = 'Theta power'
            bandwidth='auto'
        
        # filter signal using hilbert method
        filtered_sig = get_filtered_signal(raw_data, freq_range, start_cutoff, 
                                           end_cutoff, fs, bandwidth=bandwidth)
        
        # obtain power and amplitude
        filtered_sig_amp = np.real(np.abs(filtered_sig))
        filtered_sig_power = filtered_sig_amp**2
        
        # z-score data from each electrode 
        power_z = process_power(filtered_sig_power, subj_elec_sess_labels)
        
        power_z = power_z.squeeze()
        
        region_str = region_name[0]
        
        yield power_z, ylabel
        
    

In [7]:
for encoding_mode in encoding_mode_arr:
    
    # load data
    dd_trials = load_data_np(encoding_mode,
        region_name=region_name, subregion=subregion)
    
    clust = dd_trials['clust_int']
    notclust_idxs = np.argwhere(clust<0).squeeze()
    clust_idxs = np.argwhere(clust>0).squeeze()
    
    if encoding_mode:
        saveName = 'encoding_'
        recall_str = ''
        time = np.linspace(-0.7, 2.3, 150)
        correct = dd_trials['correct']
        incorrect_idxs = np.argwhere(correct==0).squeeze()
        
    else:
        saveName = 'recall_'
        recall_str = '_recall'
        time = np.linspace(-2, 2, 200)
    
    for power_z, ylabel in load_z_scored_power(dd_trials, freq_range_str_arr):

        if len(subregion[0]) > 0:
            subregion_str = f'_{subregion[0]}'
        else:
            subregion_str = ''
            
        # loop through sub-bands of gamma or theta
        for power_z_i in power_z:

            plt.figure(figsize=(10,6))

            pzc = triangleSmooth(np.mean(power_z_i[clust_idxs].squeeze(), axis=0), smoothing_triangle=5)
            pznc = triangleSmooth(np.mean(power_z_i[notclust_idxs].squeeze(), axis=0), smoothing_triangle=5)

            plt.plot(time, pzc, color='tab:blue', alpha=0.8, label="Clust")
            plt.plot(time, pznc, color='tab:orange', alpha=0.8, label="Not Clust")

            # plot not recalled data for encoding 
            if encoding_mode:
                pzi = triangleSmooth(np.mean(power_z_i[incorrect_idxs].squeeze(), axis=0), smoothing_triangle=5)
                plt.plot(time, pzi, color='tab:pink', alpha=0.8, label="Not Recalled")

            plt.axvline(0, color='black')
            plt.ylabel(f"{ylabel} ({f[0]}-{f[1]} Hz)", fontsize=18)
            plt.xlabel("Time", fontsize=18)
            sns.despine()
            plt.legend(fontsize=16)
            plt.savefig(f"{savePath}/{ylabel}_{f[0]}-{f[1]}_zscore_elec_{region_str}{subregion_str}{recall_str}", dpi=300, 
                        bbox_inches='tight')
            plt.show()

In [4]:
# now let's see which electrodes display a rise in power 
from scipy.stats import ttest_rel 
from scipy.stats import wilcoxon
       
encoding_mode = 1
fs = 500


region_name = ['HPC']
subregion = ['ca1']

freq_band = 'medium_gamma'
if freq_band == 'medium_gamma':
    freq_range = [[80,120]]
if freq_band == 'low_theta':
    freq_range = [[2,4]]
    
# load data
dd_trials = load_data_np(encoding_mode, region_name=region_name, subregion=subregion)

raw_data = dd_trials['raw']
subj_elec_sess_labels = dd_trials['elec_labels'] 
subj_elec_unique, counts_se = np.unique(subj_elec_sess_labels, return_counts=True)

if encoding_mode:

    # each trial consists of 5 sec of raw data, which starts
    # 1.7 before word onset and ends 3.3 sec after word onset 
    # the data is sampled at 500 Hz
    start_time = -700
    end_time = 2300
    
    start_cutoff = 500
    end_cutoff = 2000
    
else:
       
    # relative to word vocalization, in ms
    start_roi = -1100
    end_roi = -100

    # for recall, 
    # each trial consists of 6 sec of data, centered around word recall 
    start_time = -2000
    end_time = 2000
    
    start_cutoff = 500
    end_cutoff = 2500

bandwidth='auto'

# filter signal using hilbert method
filtered_sig = get_filtered_signal(raw_data, freq_range, start_cutoff, 
                                   end_cutoff, fs, bandwidth=bandwidth)
# obtain power and amplitude
filtered_sig_amp = np.real(np.abs(filtered_sig.squeeze()))
filtered_sig_power = decimate(np.log10(filtered_sig_amp**2), 10)

# update fs and sr_factor
fs = 50
sr_factor = 1000/fs
    

Loading data
LOADING DATA FROM:  HPC
order: C


In [29]:
# relative to word onset, in ms 
start_roi = 200
end_roi = 900
baseline_start_roi = -700
baseline_end_roi = 0

# convert to indices based on start time and sampling rate factor
start_idx = int((start_roi - start_time)/sr_factor)
end_idx = int((end_roi-start_time)/sr_factor)
baseline_start_idx = int((baseline_start_roi - start_time)/sr_factor)
baseline_end_idx = int((baseline_end_roi - start_time)/sr_factor)

print(start_idx, end_idx)
print(baseline_start_idx, baseline_end_idx)
electrode_trial_count = []
electrode_p_value_ttest = []
electrode_p_value_wilcox = []
electrode_label = []
electrode_max_time = []
for seu in subj_elec_unique:
    
    electrode_label.append(seu)
    
    se_idxs = np.argwhere(subj_elec_sess_labels==seu).squeeze()
    
    filtered_elec = filtered_sig_power[se_idxs].squeeze()
    
    max_time = int(np.mean(np.argmax(filtered_elec,axis=1)))
    
    pow_roi = np.max(filtered_elec[:, start_idx:end_idx], axis=1)
    pow_base = np.max(filtered_elec[:, baseline_start_idx:baseline_end_idx], axis=1)
    
    ttest_res = ttest_rel(pow_roi, pow_base, alternative='greater')
    
    d = np.around(pow_roi-pow_base, decimals=5)
    wilcox_res = wilcoxon(d, alternative='greater')
    
    electrode_p_value_ttest.append(ttest_res.pvalue)
    electrode_p_value_wilcox.append(wilcox_res.pvalue)
    
    electrode_trial_count.append(se_idxs.shape[0])
    

45 80
0 35


In [30]:
print("Number of electrodes: ", len(electrode_label))

Number of electrodes:  201


In [31]:
print("Number of nonsig elecs by ttest: ", np.argwhere(np.array(electrode_p_value_ttest) > 0.05).shape[0])
print("Number of nonsig elecs by wilcox: ", np.argwhere(np.array(electrode_p_value_wilcox) > 0.05).shape[0])

Number of nonsig elecs by ttest:  166
Number of nonsig elecs by wilcox:  163


In [32]:
elec_stats = pd.DataFrame({'electrode_label': electrode_label, 'pval_t_test': electrode_p_value_ttest, 'pval_wilcox': electrode_p_value_wilcox, 
                  'electrode_trial_count': electrode_trial_count})
elec_stats.to_csv(f"{savePath}/{freq_band}_elec_stats.csv")

NameError: name 'savePath' is not defined